In [ ]:
# Importar bibliotecas necessárias
import numpy as np
import pickle

var = 'f-value'
# var = 'pca'
# var = total
# Ler os dados para treino, validação e teste
# Neste caso, estamos a usar os dados que foram pré-processados utilizando o critério 'f-value' para seleção de características na pipeline feature_selection.

with open(f'X_train_validation_{var}.pkl','rb') as f:
    X_train_validation = pickle.load(f)
with open(f'y_train_validation_{var}.pkl','rb') as f:
    y_train_validation = pickle.load(f)
with open(f'X_test_{var}.pkl','rb') as f:
    X_test = pickle.load(f)
with open(f'y_test_{var}.pkl','rb') as f:
    y_test = pickle.load(f)

from sklearn.model_selection import RandomizedSearchCV

"""
Classe CustomRandomizedSearchCV:
A CustomRandomizedSearchCV é uma adaptação do RandomizedSearchCV providenciado pelo Scikit-learn.
Ela foi projetada com a intenção de simplificar e potencializar a busca aleatória de hiperparâmetros em diversos modelos de aprendizado de máquina. 
A classe foi desenvolvida para permitir uma avaliação simultânea e comparativa de diferentes modelos e as combinações de seus hiperparâmetros.

Ao concluir a pesquisa, a classe proporciona uma análise detalhada, revelando os hiperparâmetros mais eficazes para cada modelo avaliado.
Além disso, apresenta o best score de cada modelo baseando-se na métrica F1 micro-averaged.

Desta forma, otimiza-se o processo de seleção e ajuste de modelo, garantindo que os melhores hiperparâmetros
sejam identificados de maneira eficiente e sistemática.
"""

class CustomRandomizedSearchCV:
    
    def __init__(self, X, y, algorithm, parameter_distribution, cv_folds = 10, num_iters_allowed=100, score_function='f1_micro', n_jobs=25) -> None:
        self.X = X
        self.y = y
        self.algorithm = algorithm
        self.parameter_distribution = parameter_distribution
        self.cv_folds = cv_folds
        self.num_iters_allowed = num_iters_allowed
        self.score_function = score_function
        self.search = None
        self.n_jobs = n_jobs
        
    # Calcula o total de combinações de parâmetros possíveis
    def calc_total_parameters(self, params_list: list) -> int: 
        tmp = [len(lista) for lista in params_list.values()]
        res = 1
        for x in tmp:
            res *= x
        return res
    
    # Executa RandomizedSearchCV
    def random_search_cv(self):
        if self.search == None:
            percentage = float(self.num_iters_allowed) * 100 / self.calc_total_parameters(self.parameter_distribution)
            print(f"Going to train and test {self.num_iters_allowed} out of a total of {self.calc_total_parameters(self.parameter_distribution)} parameter combinations ({percentage:.2f}%)")
            random_search_cv = RandomizedSearchCV(self.algorithm, self.parameter_distribution, random_state=0, n_iter=self.num_iters_allowed, cv=self.cv_folds, n_jobs=self.n_jobs, scoring=self.score_function)
            self.search = random_search_cv.fit(self.X, self.y)
    
    # Retorna os melhores parâmetros encontrados
    def get_best_parameters(self):
        self.random_search_cv()
        return self.search.best_params_
    
    def get_best_score(self):
        self.random_search_cv()
        return self.search.best_score_
    
    def get_average_train_time(self):
        self.random_search_cv()
        return np.mean(self.search.cv_results_['mean_fit_time'])
    
    def get_average_test_time(self):
        self.random_search_cv()
        return np.mean(self.search.cv_results_['mean_score_time'])
    
    def get_best_estimator(self):
        self.random_search_cv()
        return self.search.best_estimator_
    
    def get_all_cv_results(self):
        self.random_search_cv()
        return self.search.cv_results_

    def print_details(self):
        print(f"Best score: {self.get_best_score()}")
        print(f"Best parameters: {self.get_best_parameters()}")
        print(f"Average train time (secs): {self.get_average_train_time():.2}")
        print(f"Average test time (secs): {self.get_average_test_time():.2}")

from sklearn.svm import SVC
# O algoritmo SVM é útil para encontrar fronteiras não lineares complexas em conjuntos de dados de alta dimensão.

svm_random_search_cv = CustomRandomizedSearchCV(X_train_validation, y_train_validation, algorithm=SVC(class_weight='balanced'),
                               parameter_distribution=dict(C=np.logspace(np.log10(0.1),np.log10(1000), 1000), 
                                                           kernel=['linear', 'poly', 'rbf'], 
                                                           gamma=np.logspace(np.log10(0.001),np.log10(10), 1000), 
                                                           max_iter=[100, 200, 500, 1000]),
                               num_iters_allowed=1000)

svm_random_search_cv.print_details()


from sklearn.neighbors import KNeighborsClassifier
# KNN é um algoritmo de aprendizagem baseado em instância e é usado tanto para classificação quanto regressão.

knn_random_search_cv = CustomRandomizedSearchCV(X_train_validation, y_train_validation, algorithm=KNeighborsClassifier(),
                               parameter_distribution=dict(n_neighbors=[int(x) for x in np.linspace(1, 30, 30)], 
                                                           algorithm=['ball_tree', 'kd_tree', 'brute'],
                                                           weights=['uniform', 'distance'], 
                                                           metric=['euclidean', 'manhattan', 'minkowski']), 
                               num_iters_allowed=1000)


knn_random_search_cv.print_details()


from sklearn.linear_model import LogisticRegression
# A regressão logística é usada quando a variável dependente é categórica.

log_reg_random_search_cv = CustomRandomizedSearchCV(X_train_validation, y_train_validation, algorithm=LogisticRegression(),
                               parameter_distribution=dict(penalty=['l1', 'l2', 'elasticnet', 'none'],
                                                           solver=['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
                                                           max_iter=[100, 200, 500, 1000, 2000, 3500, 5000],
                                                           multi_class=['auto', 'ovr', 'multinomial'],
                                                           C=np.linspace(0.5, 2, 30)), 
                               num_iters_allowed=1000)


log_reg_random_search_cv.print_details()


from sklearn.tree import DecisionTreeClassifier
# As árvores de decisão são usadas na tomada de decisões e para resolver problemas de classificação e regressão.

dt_random_search_cv = CustomRandomizedSearchCV(X_train_validation, y_train_validation, algorithm=DecisionTreeClassifier(),
                               parameter_distribution=dict(
                               criterion=['gini', 'entropy'],
                               splitter=['best', 'random'],
                               max_depth=[None, 2, 4, 8, 16],
                               min_samples_split=[2, 4, 8, 16, 32],
                               # min_samples_leaf=[1],
                               max_features=['auto', 'sqrt', 'log2'],
                               ), 
                               num_iters_allowed=1000)


dt_random_search_cv.print_details()

from sklearn.ensemble import BaggingClassifier


from sklearn.ensemble import RandomForestClassifier
# Um random forest é um meta-estimador que se ajusta a uma série de árvores de decisão.

rf_random_search_cv = CustomRandomizedSearchCV(X_train_validation, y_train_validation, algorithm=RandomForestClassifier(n_estimators=100),
                               parameter_distribution=dict(
                                   criterion=['gini', 'entropy'],
                                    max_depth=[None, 2, 4, 8, 16],
                                    min_samples_split=[2, 4, 8, 16, 32],
                                    max_features=['auto', 'sqrt', 'log2'],
                               ), 
                               num_iters_allowed=1000)

rf_random_search_cv.print_details()

from sklearn.ensemble import AdaBoostClassifier
# AdaBoost é um algoritmo de boosting

ada_b_random_search_cv = CustomRandomizedSearchCV(X_train_validation, y_train_validation, algorithm=AdaBoostClassifier(),
                               parameter_distribution=dict(algorithm=['SAMME', 'SAMME.R'],
                                                           n_estimators=[10, 50, 100, 200],
                                                           learning_rate= np.logspace(np.log10(0.01),np.log10(1), 100)
                                                           
                               ), 
                               num_iters_allowed=1000)


ada_b_random_search_cv.print_details()

from sklearn.neural_network import MLPClassifier
# MLP é uma rede neural artificial que pode ser usada tanto para classificação quanto para regressão.

mlp_single_layer_random_search_cv = CustomRandomizedSearchCV(X_train_validation, y_train_validation, algorithm=MLPClassifier(),
                               parameter_distribution=dict(hidden_layer_sizes=[(128,),
                                                                               (256,),
                                                                               (512,),
                                                                               (1024,),
                                                                               (2048,)],
                                                           solver=['lbfgs', 'sgd', 'adam'],
                                                           learning_rate=['constant', 'invscaling', 'adaptive'],
                                                           learning_rate_init=np.logspace(np.log10(0.001),np.log10(1), 1000),
                                                           max_iter=[200, 500, 2000],
                                                           batch_size=["auto", 64, 512]
                                                           ), 
                               num_iters_allowed=1000,
                               )



mlp_single_layer_random_search_cv.print_details()

from sklearn.neural_network import MLPClassifier

# algorithm = MLPClassifier()
mlp_dnn_random_search_cv = CustomRandomizedSearchCV(X_train_validation, y_train_validation, algorithm=MLPClassifier(),
                               parameter_distribution=dict(hidden_layer_sizes=[(512, 256, 128, 128, 128, 128, 128, 128),
                                                                               (256, 128, 128, 128, 128, 128, 128, 128),
                                                                               (512, 256, 128, 128, 128, 128, 128),
                                                                               (512, 256, 128, 128, 128, 128),
                                                                               (256, 128, 128, 128, 128), 
                                                                               (512, 128, 128, 128, 128), 
                                                                               (512, 128, 128, 128),
                                                                               (256, 128, 128, 128),
                                                                               (256, 128, 128), 
                                                                               (512, 128),
                                                                               (256, 128)],
                                                           solver=['lbfgs', 'sgd', 'adam'],
                                                           learning_rate=['constant', 'invscaling', 'adaptive'],
                                                           learning_rate_init=np.logspace(np.log10(0.001),np.log10(1), 1000),
                                                           max_iter=[200, 500, 2000],
                                                           batch_size=["auto", 64, 512]
                                                           ), 
                               num_iters_allowed=1000,
                               )

mlp_dnn_random_search_cv.print_details()
